In [1]:
import spacy
import numpy as np
import pandas as pd
from stopwords import ENGLISH_STOP_WORDS
import warnings
from __future__ import unicode_literals
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
#en_nlp = spacy.en.English()
en_nlp=spacy.load('en')

In [3]:
def spacy_get_vec(sentence):
    vec = np.zeros(384)
    doc = en_nlp((sentence))
    for word in doc:
        #if word.lower_ in ENGLISH_STOP_WORDS:
        #    continue
        vec += word.vector
    return vec

In [4]:
lines = open('./question.txt').readlines()
vecs = []
intents = []
idfs = []
for line in lines:
    tokens = line.split(',')
    sentence = tokens[0]
    intent = tokens[1]
    if intent[-1] == '\n':
        intent = intent[:-1]
    vecs.append(spacy_get_vec(sentence))
    intents.append(intent)
    #idfs.append(get_idf(sentence))

df = pd.DataFrame(vecs, columns=['vec_%d' % i for i in range(384)])
#df['idf'] = idfs
df['intents'] = intents
df.intents = df.intents.astype('category')

In [5]:
df.head(10)

,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_375,vec_376,vec_377,vec_378,vec_379,vec_380,vec_381,vec_382,vec_383,intents
0,-3.543652,4.848562,4.919341,13.844563,5.203950,-0.971813,-2.271860,2.404070,4.035475,6.618456,...,0.278471,0.737995,-0.844568,0.292816,1.108275,-0.625411,0.886418,1.066385,1.134127,name
1,5.643370,-1.788185,11.669976,5.293034,4.905268,2.301256,3.591100,-3.610936,-0.577742,3.764210,...,0.364541,0.103859,-0.135133,0.627274,1.794037,-0.896731,1.003043,0.172758,0.383363,name
2,-1.656146,-3.490586,9.790032,7.341413,5.708977,2.287670,1.429370,-1.337799,3.224365,3.973423,...,0.381141,0.415805,-0.634530,0.283914,1.596852,-0.837435,0.489029,1.138625,0.622295,name
3,1.293344,3.555448,12.452135,5.784851,-2.995595,10.809371,-4.327154,-0.921649,2.327775,8.620191,...,0.787688,0.961237,-0.608474,0.349756,1.407382,-0.791489,0.703010,2.186781,0.517962,name
4,4.153239,6.531323,13.412330,2.088275,5.419556,2.024746,-2.581261,-3.334134,0.208024,6.853853,...,1.340130,0.999525,-1.148018,0.732091,1.861112,-0.490642,0.414732,1.869674,1.112716,name
5,7.831067,-2.849134,12.220646,4.811266,6.354322,-1.116443,0.997382,-7.557665,0.285819,-0.831057,...,0.112487,0.110709,-0.479749,0.964044,2.495846,-0.488649,0.403862,1.576550,-0.271851,name
6,0.831167,-1.640797,10.277429,7.369606,5.822305,2.301420,3.385551,-0.424612,2.688910,0.313156,...,0.174171,0.535187,-0.426263,-0.461186,1.373420,0.006966,1.253729,1.917415,1.136800,name
7,3.870726,-0.750165,8.419325,0.993593,4.111084,-1.089855,-1.959931,-3.264334,2.925440,2.240141,...,0.908486,0.034517,-0.306704,0.796868,1.662183,-1.061653,0.973119,0.577741,0.380746,name
8,-1.656146,-3.490586,9.790032,7.341413,5.708977,2.287670,1.429370,-1.337799,3.224365,3.973423,...,0.381141,0.415805,-0.634530,0.283914,1.596852,-0.837435,0.489029,1.138625,0.622295,name
9,7.717528,-0.098212,7.097780,13.344487,-5.314187,9.025711,-1.001992,-4.040305,8.524757,-1.291224,...,0.472477,0.061492,-0.604970,0.359515,1.386781,-0.042573,1.397350,1.775708,-0.164133,name


In [6]:
from sklearn.utils import shuffle
df = shuffle(df)

In [7]:
X = df.iloc[:, :-1].values
y = df.iloc[:,-1:].values.ravel()

In [8]:
from sklearn.cross_validation import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.20)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
from sklearn.linear_model import LogisticRegression
logit_model = LogisticRegression()
logit_model.fit(X_train, y_train)
print(logit_model.score(X_val, y_val))


0.833333333333


In [10]:
from sklearn.ensemble import GradientBoostingClassifier
gradboost = GradientBoostingClassifier(n_estimators=600, max_depth=30, learning_rate=0.01)
gradboost.fit(X_train, y_train)
print(gradboost.score(X_val, y_val))


0.666666666667


In [11]:
sent = 'do you live in the computer'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[ -8.81953237e+00  -8.81157917e+00  -7.96046149e-04  -8.68235072e+00
   -8.70572591e+00  -8.71730751e+00]]
[u'location']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [12]:
sent = 'how does your mom call you'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[ -9.17711136e+00  -9.16915816e+00  -9.02683503e+00  -9.04073123e+00
   -5.60872124e-04  -9.07488650e+00]]
[u'name']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [13]:
sent = 'how does my mom call me'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[-9.39218666 -9.38423345 -9.24191033 -9.21899022 -1.33852408 -0.30461918]]
[u'users_name']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [14]:
gradboost.classes_

array([u'bot', u'greet_response', u'location', u'marriage', u'name',
       u'users_name'], dtype=object)

In [15]:
from sklearn.svm import LinearSVC
svcmodel=LinearSVC(multi_class='ovr')
svcmodel.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class=u'ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [16]:
svcmodel.score(X_val,y_val)

0.83333333333333337

In [17]:
sent = 'do you live in the computer'
#print(svcmodel.predict_log_proba(spacy_get_vec(sent)))
print(svcmodel.predict(spacy_get_vec(sent)))

[u'location']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [18]:
sent = 'how does your mom call you'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[[ -9.17711136e+00  -9.16915816e+00  -9.02683503e+00  -9.04073123e+00
   -5.60872124e-04  -9.07488650e+00]]
[u'name']


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [19]:
from sklearn.naive_bayes import MultinomialNB
mnb= MultinomialNB(alpha=1.7)
mnb.fit(abs(X_train), y_train)

MultinomialNB(alpha=1.7, class_prior=None, fit_prior=True)

In [20]:
mnb.score(abs(X_val), y_val)

0.91666666666666663

In [21]:
svcmodel.fit(X,y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class=u'ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [22]:
from sklearn.externals import joblib
joblib.dump(svcmodel, 'question.pkl')

[u'question.pkl']